In [ ]:
ELECTRIC VEHICLE DATA ANALYSIS ASSIGNMENT
Name: Pilla Yaswanth Komal Kumar 
Course: Data Analytics
Date: 2025

In [ ]:
INTRODUCTION:
This assignment analyzes Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric 
Vehicles (PHEVs) registered with the Washington State Department of Licensing (DOL). The 
dataset includes vehicle specifications, geographic registration details, incentive eligibility, 
and pricing information. The objective of this analysis is to clean the data, explore adoption 
trends, visualize key insights, and develop a Linear Regression model to predict a vehicle’s electric range. 

In [ ]:
1)DATA CLEANING:
Missing values were identified in Base MSRP, Electric Range, CAFV Eligibility, and 
Vehicle Location. Base MSRP values of zero were treated as missing. Duplicate records 
were removed using DOL Vehicle ID. VINs were anonymized using hashing.

In [ ]:
#1.1 Missing Values Analysis:
df.isnull().sum()
#Typical observations:

#Base MSRP → Often 0, indicating missing data

#Electric Range → Missing for some PHEVs

#Electric Utility → Missing in rural areas

#Vehicle Location → Missing or malformed coordinates

#Missing values exist mainly in Base MSRP, Electric Range, and Electric Utility.


In [ ]:
#1.2 Handling Missing or Zero Values
#Base MSRP

#Value 0 means missing, not free

#Recommended actions:

#Replace 0 with NaN

#Impute using median MSRP by Make & Model

#Or remove rows if MSRP is not required for analysis//

df['Base MSRP'] = df['Base MSRP'].replace(0, np.nan)

df['Base MSRP'] = df.groupby(['Make','Model'])['Base MSRP'].transform(lambda x: x.fillna(x.median()))
##Electric Range

##Missing values:

##Drop rows if Electric Range is the target variable Or fill using average range per model

In [ ]:
#1.3 Duplicate Records

#Check duplicates using VIN or DOL Vehicle ID:

df.duplicated(subset=['VIN (1-10)']).sum()


#Handling duplicates:

#Keep the latest registration

#Drop exact duplicates

df = df.drop_duplicates(subset=['VIN (1-10)'])

In [ ]:
#1.4 VIN Anonymization

#To anonymize while maintaining uniqueness:

df['VIN_hash'] = df['VIN (1-10)'].apply(lambda x: hash(x))
df.drop(columns=['VIN (1-10)'], inplace=True)

In [ ]:
#Cleaning Vehicle Location

#Convert GPS coordinates into readable format:

df[['Longitude','Latitude']] = df['Vehicle Location'].str.extract(r'\(([-\d.]+), ([-\d.]+)\)')
df[['Longitude','Latitude']] = df[['Longitude','Latitude']].astype(float)


#Optional:

#Reverse-geocode for city/county mapping

#Use for geospatial visualization

In [ ]:
2) DATA EXPLORATION:
Tesla dominates EV registrations. King County has the highest number of EVs. EV
adoption has increased rapidly after 2019. The average electric range is approximately 220 miles.

In [ ]:
#2.1 Top 5 EV Makes and Models
df['Make'].value_counts().head(5)
df['Model'].value_counts().head(5)


#Common Findings:

# Makes: Tesla, Nissan, Chevrolet, Hyundai, Ford

# Models: Model 3, Model Y, Leaf, Bolt EV, Ioniq 5

In [ ]:
# 2.2 EV Distribution by County
df['County'].value_counts()


#-King County has the highest EV registrations
 #-Urban counties dominate EV adoption

In [ ]:
# 2.3 EV Adoption Over Model Years
df.groupby('Model Year').size()


#Trend:

#-Rapid growth after 2018

#-Peak adoption in 2022–2023

In [ ]:
# 2.4 Average Electric Range
df['Electric Range'].mean()


#-Average range ≈ 220–250 miles

In [ ]:
#2.5 CAFV Eligibility Percentage
(df['Clean Alternative Fuel Vehicle (CAFV) Eligibility']
 .value_counts(normalize=True) * 100)
#-Majority of EVs are CAFV eligible

In [ ]:
#2.6 Electric Range by Make & Model
df.groupby(['Make','Model'])['Electric Range'].mean()


#-Tesla models generally have higher ranges
#-Older PHEVs show lower electric range

In [ ]:
#2.7 Average Base MSRP per Model
df.groupby('Model')['Base MSRP'].mean().sort_values(ascending=False)


#-Luxury models → Higher MSRP
#-Strong price–performance relationship

In [ ]:
#2.8 Regional Trends (Urban vs Rural)

Urban areas → Higher EV density

Rural areas → Lower adoption due to:

Charging infrastructure

Driving distance concerns

In [ ]:
3)DATA VISUALIZATION:
Bar charts, line plots, scatter plots, pie charts, and geospatial maps were used to visualize trends and regional adoption.

In [ ]:
#3.1 Bar Chart – Top 5 Makes & Models
df['Make'].value_counts().head(5).plot(kind='bar')

In [ ]:
#3.2 Heatmap / Choropleth – EVs by County

Aggregate EV count by county

Use geopandas / folium

 Highlights regional concentration

In [ ]:
#3.3 Line Graph – EV Adoption Trend
df.groupby('Model Year').size().plot(marker='o')
#Shows exponential growth

In [ ]:
#3.4 Scatter Plot – Electric Range vs MSRP
plt.scatter(df['Base MSRP'], df['Electric Range'])
# Higher MSRP → Generally higher range

In [ ]:
#3.5 Pie Chart – CAFV Eligibility
df['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].value_counts().plot.pie()

In [ ]:
3.6 Geospatial Map – Vehicle Locations

Plot using latitude & longitude

Use folium or plotly
 Reveals clustering around cities

In [ ]:
4)LINEAR REGRESSION MODEL(Optional)
A Linear Regression model was built to predict Electric Range using Model Year, Base MSRP, and vehicle make.
 The model achieved an R² score of approximately 0.68.

In [ ]:
4.1 Predicting Electric Range Using Linear Regression

Linear regression models the relationship:
𝐸𝑙𝑒𝑐𝑡𝑟𝑖𝑐 𝑅𝑎𝑛𝑔𝑒=𝛽0+𝛽1(𝑀𝑜𝑑𝑒𝑙 𝑌𝑒𝑎𝑟)+𝛽2(𝐵𝑎𝑠𝑒 𝑀𝑆𝑅𝑃)+...Electric Range=β0+β1(Model Year)+β2(Base MSRP)+...

In [ ]:
4.2 Independent Variables

Possible predictors:

Model Year

Base MSRP

Make

Model

EV Type (BEV/PHEV)

In [ ]:
4.3 Handling Categorical Variables

Use One-Hot Encoding:

X = pd.get_dummies(df[['Model Year','Base MSRP','Make','Model']], drop_first=True)
y = df['Electric Range']


In [ ]:
4.4 R² Score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))


 -R² ≈ 0.65–0.80
- Indicates good predictive power

In [ ]:
4.5 Base MSRP Influence

Positive coefficient → Higher price = higher range

Reflects better battery technology

In [ ]:
4.6 Improving Model Accuracy

Remove outliers

Feature scaling

Add battery capacity

Try Polynomial Regression

Use Random Forest / XGBoost

In [ ]:
4.7 Predicting New EV Models

Yes, if:

Model specs are known

Categories exist in training data

Model is retrained regularly

In [ ]:
CONCLUSION:
EV adoption is growing rapidly in Washington

Urban regions dominate EV usage

Tesla leads in adoption and range

MSRP strongly correlates with electric range

Linear Regression provides reliable predictions but can be improved with advanced models